In [293]:
pip install category_encoders

In [294]:
import pandas as pd
data2=pd.read_csv('Cleaned_Bengaluru_House_Data.csv')
print(data2.shape)

(5887, 9)


In [295]:
data2.drop(columns=['price_per_sqft','availability'], inplace=True)


In [296]:
data2.head()

,area_type,location,size(BHK),total_sqft,bath,balcony,price
0,Super built-up Area,1st Phase JP Nagar,3.0,1875.0,3.0,1.0,167.0
1,Super built-up Area,1st Phase JP Nagar,2.0,1394.0,2.0,1.0,100.0
2,Super built-up Area,1st Phase JP Nagar,2.0,1077.0,2.0,2.0,93.0
3,Built-up Area,1st Phase JP Nagar,1.0,1280.0,1.0,1.0,63.0
4,Built-up Area,1st Phase JP Nagar,1.0,840.0,2.0,2.0,50.0


In [297]:
x=data2.drop(columns=['price'])
y=data2['price']

In [298]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from category_encoders import BinaryEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

**LinearRegression**

In [299]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=0)

In [300]:
from category_encoders import TargetEncoder

column_transf = make_column_transformer(
    (TargetEncoder(), ['location']),
    (OneHotEncoder(sparse_output=False, handle_unknown='ignore'),['area_type']),remainder='passthrough')


In [301]:
scalar=StandardScaler()

In [302]:
lr=LinearRegression()

In [303]:
pipe=make_pipeline(column_transf,scalar,lr)

In [304]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('targetencoder',
                                                  TargetEncoder(),
                                                  ['location']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['area_type'])])),
                ('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [305]:
y_pridict_lr=pipe.predict(X_test)

In [306]:
r2 = r2_score(y_test, y_pridict_lr)
print("R² score:", r2)

R² score: 0.8277192291063264


In [307]:
cv_scores = cross_val_score(pipe, x, y, cv=5, scoring='r2')
print("Cross-validated R² scores for each fold:", cv_scores)
print("Average cross-validated R² score:", np.mean(cv_scores))

Cross-validated R² scores for each fold: [0.6757895  0.72067829 0.74601233 0.69683269 0.7542557 ]
Average cross-validated R² score: 0.7187137012880025


**LassoRegression**

In [308]:
lasso=Lasso()

In [309]:
pipe1=make_pipeline(column_transf,scalar,lasso)

In [310]:
pipe1.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('targetencoder',
                                                  TargetEncoder(),
                                                  ['location']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['area_type'])])),
                ('standardscaler', StandardScaler()), ('lasso', Lasso())])

In [311]:
y_pridict_lasso=pipe1.predict(X_test)
r2 = r2_score(y_test, y_pridict_lasso)
print("R² score:", r2)


R² score: 0.8244182682969603


In [312]:
cv_scores = cross_val_score(pipe1, x, y, cv=10, scoring='r2')
print("Cross-validated R² scores for each fold:", cv_scores)
print("Average cross-validated R² score:", np.mean(cv_scores))

Cross-validated R² scores for each fold: [0.72447109 0.60945276 0.67183283 0.71150825 0.77967338 0.7033204
 0.75472862 0.73852508 0.74425457 0.75726416]
Average cross-validated R² score: 0.7195031139045147


**RidgeRegression**

In [313]:
ridge = Ridge()


In [314]:
pipe2 = make_pipeline(column_transf, scalar, ridge)

In [315]:
pipe2.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('targetencoder',
                                                  TargetEncoder(),
                                                  ['location']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['area_type'])])),
                ('standardscaler', StandardScaler()), ('ridge', Ridge())])

In [316]:
y_predict_ridge = pipe2.predict(X_test)
r2 = r2_score(y_test,y_predict_ridge)
print("R² score:", r2)

R² score: 0.8279563084753038


In [317]:
cv_scores = cross_val_score(pipe2, x, y, cv=10, scoring='r2')
print("Cross-validated R² scores for each fold:", cv_scores)
print("Average cross-validated R² score:", np.mean(cv_scores))

Cross-validated R² scores for each fold: [0.72565798 0.6187706  0.67430336 0.72144624 0.77852994 0.7072802
 0.77379558 0.75096852 0.74665214 0.75977953]
Average cross-validated R² score: 0.7257184101617321


In [318]:
from xgboost import XGBRegressor
xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=40)



In [319]:
pipe3 = make_pipeline(column_transf, scalar, xgb)

In [320]:
pipe3.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('targetencoder',
                                                  TargetEncoder(),
                                                  ['location']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['area_type'])])),
                ('standardscaler', StandardScaler()),
                ('xgbregressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsamp...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=100, n_jobs=None,
                              num_parallel_tree=None, random_state=40, ...))])

In [321]:
y_predict_xgb = pipe3.predict(X_test)
r2 = r2_score(y_test,y_predict_xgb)
print("R² score:", r2)

R² score: 0.8681583587175938


In [322]:
import pickle
pickle.dump(pipe3,open('XGB_Model.pkl','wb'))

In [323]:
import pickle
pickle.dump(pipe2,open('RidgeModel.pkl','wb'))